In [1]:
# y --> 0 0r 1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

In [2]:
data = pd.read_csv("banking.csv")
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,44,blue-collar,married,basic.4y,unknown,yes,no,cellular,aug,thu,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,0
1,53,technician,married,unknown,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.021,5195.8,0
2,28,management,single,university.degree,no,yes,no,cellular,jun,thu,...,3,6,2,success,-1.7,94.055,-39.8,0.729,4991.6,1
3,39,services,married,high.school,no,no,no,cellular,apr,fri,...,2,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,0
4,55,retired,married,basic.4y,no,yes,no,cellular,aug,fri,...,1,3,1,success,-2.9,92.201,-31.4,0.869,5076.2,1


In [ ]:
data.info()
data['y'].value_counts()

In [ ]:
%matplotlib inline
table = pd.crosstab(data['education'],data['y']).plot(kind='bar')
plt.title("purchase")
plt.xlabel('job')
plt.ylabel('Fre of purchase')

In [17]:
data_fin = pd.get_dummies(data,drop_first='True')
data_fin['y'].value_counts(normalize=True)

# Smote
X = data_fin.drop(columns='y')
y = data_fin['y']

y
0    0.887346
1    0.112654
Name: proportion, dtype: float64

In [ ]:
from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns

os_data_X,os_data_y=os.fit_resample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['y'])

In [ ]:
# Recursive Feature Elimination technique
from sklearn.feature_selection import RFE

logreg = LogisticRegression()
rfe = RFE(logreg, n_features_to_select=20)
rfe = rfe.fit(os_data_X, os_data_y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
cols = X.columns[rfe.support_]

X=os_data_X[cols]
y=os_data_y['y']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

In [ ]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

In [19]:
#Evaluation 
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix,f1_score,recall_score,precision_score


In [ ]:
cm = confusion_matrix(y_test,y_pred)
cm_display=ConfusionMatrixDisplay(cm, display_labels=[0,1])
cm_display.plot()
plt.show()

In [ ]:
print(f"precision:{precision_score(y_test,y_pred)}")
#same for recall and f1 score